In [2]:
import pandas as pd
from tqdm import tqdm 

In [13]:
print('start!')
train_df = pd.read_csv('/home/wjunneng/Ubuntu/2020-AI-Know-The-Text-Summary/data/input/train.csv')
test_df = pd.read_csv('/home/wjunneng/Ubuntu/2020-AI-Know-The-Text-Summary/data/input/test.csv')

start!


In [7]:
train_df.head()

,article,summarization
0,南方都市报2015-07-30<Paragraph>17:37南都讯<Paragraph>记...,城市非主干道两侧将临时指定路段时段供食品摊贩经营，黄金地段摆摊需摇号，食品摊贩无需工商登记
1,据湖北省纪委消息：近期，经湖北省委批准，湖北省纪委对鄂州市人大常委会原党组书记、主任涂维发严...,鄂州市人大常委会主任涂维发被双开；经查，其利用职务便利为他人谋利，收受贿赂，涉违法违纪。
2,［球心］19:54西汉姆联队首发：阿德里安，奥邦纳，雷德，汤姆金斯，克雷斯维尔，奥克斯福德，...,英超首轮：阿森纳主场0-2不敌西汉姆联，切赫登陆阿森纳首秀不顺，城门两次宣告失守
3,希腊司法部长尼科斯·帕拉斯基沃普洛斯11日说，他准备签署文件，允许政府没收德国在希腊资产，以...,希腊拟没收境内德国财产，以偿还德国所欠二战赔款；德国称已完成向二战受害国的赔偿事宜，希腊索赔...
4,中新网11月25日电<Paragraph>据外交部网站消息，2013年11月25日，外交部部...,外交部召见日本驻华大使 就日方无理指责划设东海防空识别区提出严正抗议


In [9]:
from textrank4zh import TextRank4Keyword,TextRank4Sentence

In [20]:
texts = test_df['article'].values
res = []
def deal_text(text: str):
    """
    处理文本
    """
    text = text.replace('<Paragraph>', ' ')

    for special_char in ['■', '●', '\t', '\n']:
        text = text.replace(special_char, ' ')

    for (ch_char, en_char) in zip(['０', '１', '２', '３', '４', '５', '６', '７', '８', '９'],
                                  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']):
        text = text.replace(ch_char, en_char)

    return text

for text in tqdm(texts):
    text = deal_text(text)
    tr4s = TextRank4Sentence()
    tr4s.analyze(text=text, lower=True, source = 'no_stop_words')
    try:
        key_sentences = tr4s.get_key_sentences(num=1, sentence_min_len=30)        
        res.append(key_sentences[0]['sentence'])
    except:
        key_sentences = tr4s.get_key_sentences(num=1, sentence_min_len=20)        
        res.append(key_sentences[0]['sentence'])

100%|██████████| 10000/10000 [18:13<00:00,  9.14it/s] 


In [21]:
sub = pd.DataFrame()
sub['sum'] = res
sub['sum'] = sub['sum'].apply(lambda a: a+'。')
sub.head()

,sum
0,发布日期：2015-05-20 08:37:00龙岩市气象台5月20日8时37分变更发布暴雨...
1,妻子徐某站在旁边，不劝说丈夫也就算了，还将对方赔偿的六百元现金拿出来甩给女服务员母亲，“这钱...
2,阿雷西博天文台表示，大部分小行星是众多较小的岩石，受重力影响松散地聚在一起，而UW-158形...
3,汝州市小屯镇朝川村11岁男孩儿洋洋（化名），因喉部肿痛在学校打电话给妈妈让带去看医生，洋洋被...
4,"图为5月12日，上海，黄川即将被推入手术室,他安慰妻子,“我进去睡一觉就出来了,别担心”。"


In [22]:
sub.to_csv('/home/wjunneng/Ubuntu/2020-AI-Know-The-Text-Summary/data/output/sub.csv',header=None)